In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

In [ ]:
# Custom functions
from useful_functions import get_inventory_dataset, init_simple_lca, multi_lcia, multi_contribution_analysis, calculate_projected_impacts
from visualisation_functions import plot_multilca_impacts, plot_contribution_analysis, plot_impacts_with_production, plot_incremental_impacts

# Set projects, LCI and LCIA methods

In [ ]:
BW_PROJECT = 'lib_rm' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

In [ ]:
EI_DB = 'ecoinvent-3.10-cutoff' # name of ecoinvent database in your project
LIB_RM_DB = "LIB raw materials"

In [ ]:
# Import IW+2.1 for ecoinvent 3.10 using the footprint version
footprint_ei310 = 'data/IW+2.1/impact_world_plus_21_brightway2_footprint_version_ei310.a7763e0d1b0d9263f49a7021cab8ef03.bw2package'
bw.BW2Package.import_file(footprint_ei310)

In [ ]:
# Search for activities
# [a["name"] for a in bd.Database('LIB raw materials')]

In [ ]:
#activity = bd.Database('ecoinvent-3.10-cutoff').search('copper mine operation and beneficiation, sulfide ore')
#selected_activity = [act for act in activity if act['location'] == 'CA']

In [ ]:
INVENTORIES_EI = {
    ## Neodymium
    "Neodymium":        ("rare earth oxides production, from rare earth carbonate concentrate", "neodymium oxide", "RoW"),
    
    ## Copper
    "Copper concentrate, Canada": ('copper mine operation and beneficiation, sulfide ore', 'copper concentrate, sulfide ore', 'CA'),
    "Market for copper, cathode": ('market for copper, cathode', 'copper, cathode', 'GLO')
}

In [ ]:
# From Istrate et al (2024)
# (name, reference product, location)
INVENTORIES = {
    "Lithium hydroxide, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium hydroxide, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
    "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
    
    "Lithium carbonate, brine":        ("lithium carbonate production, Salar de Atacama", "lithium carbonate, battery grade", "CL"),
    "Lithium carbonate, spodumene":    ("lithium carbonate production, from Australian spodumene", "lithium carbonate, battery grade", "CN"),
}

In [ ]:
# Collect data from both the custom database and Ecoinvent database
INVENTORIES_ds = {}
INVENTORIES_ds.update(get_inventory_dataset(INVENTORIES, LIB_RM_DB))
INVENTORIES_ds.update(get_inventory_dataset(INVENTORIES_EI, EI_DB))

In [ ]:
INVENTORIES_ds

In [ ]:
bd.methods

In [ ]:
# Filter and display methods that contain "IMPACT World+" in their names
iw_methods = [method for method in bd.methods if "impact world" in " ".join(method).lower()]
iw_methods

In [ ]:
#bd.methods
IMPACT_METHODS = {
    "Carbon footprint": ('IMPACT World+ Footprint 2.1 for ecoinvent v3.10', 'Carbon footprint'), 
    "Energetic resource depletion": ('IMPACT World+ Footprint 2.1 for ecoinvent v3.10', 'Energetic resource depletion'),
    "Water footprint": ('IMPACT World+ Footprint 2.1 for ecoinvent v3.10', 'Water footprint - Scarcity'),
    "Human health (residual)": ('IMPACT World+ Footprint 2.1 for ecoinvent v3.10', 'Human health (residual)'),
    "Ecosystem quality (residual)": ('IMPACT World+ Footprint 2.1 for ecoinvent v3.10', 'Ecosystem quality (residual)'),
                 }

In [ ]:
impact_categories = [
    "Carbon footprint (kg CO2-Eq)",
    "Energetic resource depletion (MJ deprived)",
    "Water footprint (m3 world-eq)", 
    "Human health (DALY)",
    "Ecosystem quality (PDF.m2.yr)", 
]

In [ ]:
#bd.methods
IMPACT_METHODS = {
    "Climate change": ('IMPACT World+ v2.0.1, footprint version', 'climate change', 'carbon footprint'), 
    "Ecosystem quality": ('IMPACT World+ v2.0.1, footprint version', 'ecosystem quality', 'remaining ecosystem quality damage'),
    "Human health": ('IMPACT World+ v2.0.1, footprint version', 'human health', 'remaining human health damage'),
    "Water use": ('IMPACT World+ v2.0.1, footprint version', 'water use', 'water scarcity footprint')
                 }

In [ ]:
impact_categories = [
    "Climate change (kg CO2-Eq)", 
    "Ecosystem quality (PDF.m2.yr)", 
    "Human health (DALYs)", 
    "Water use (m3 world Eq deprived)"
]

# Specific impacts

In [ ]:
# Initialize LCA object
lca = init_simple_lca(INVENTORIES_ds["Lithium hydroxide, brine"], method=IMPACT_METHODS )

In [ ]:
specific_lca = {}
for rm in INVENTORIES_ds:
    impacts = multi_lcia(lca, INVENTORIES_ds[rm], IMPACT_METHODS)
    specific_lca[rm] = impacts

specific_lca = pd.DataFrame(specific_lca).T

In [ ]:
specific_lca

In [ ]:
# Call the function
plot_multilca_impacts(specific_lca, 
                 impact_categories = [impact_categories], 
                 colors=["#264653", "#2a9d8f", "#e9c46a", "#f4a261", "e76f51"], 
                 save_path="results/lca_impacts.png")

In [ ]:
specific_lca.to_csv(f"results/specific_results_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Raw material")

In [ ]:
# Initialize a dictionary to store contribution results for each inventory
contribution_results = {}

# Loop through each inventory and perform contribution analysis
for rm_name, rm_ds in INVENTORIES_ds.items():
    lca = init_simple_lca(rm_ds)
    contributions = multi_contribution_analysis(lca, IMPACT_METHODS, top_n=10, threshold=0.01)  # Set your threshold here
    contribution_results[rm_name] = contributions

# Convert the results into a more readable format for analysis
contribution_dfs = {}

for rm_name, impacts in contribution_results.items():
    for impact_name, contributions in impacts.items():
        df = pd.DataFrame(contributions)
        df["Inventory"] = rm_name
        df["Impact Category"] = impact_name
        contribution_dfs[(rm_name, impact_name)] = df

# Combine all individual DataFrames into one for easy viewing
contribution_analysis_df = pd.concat(contribution_dfs.values(), ignore_index=True)

# Set "Inventory" and "Impact Category" as the row indices
contribution_analysis_df.set_index(["Inventory", "Impact Category"], inplace=True)


In [ ]:
contribution_analysis_df

In [ ]:
# Example usage
inventory_names = ["Lithium hydroxide, brine", "Lithium hydroxide, spodumene", "Cobalt"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]
plot_contribution_analysis(contribution_analysis_df, inventory_names, colors, save_dir="results")

In [ ]:
contribution_analysis_df.to_csv(f"results/contribution_analysis_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

# Demand-related impacts

In [ ]:
production_existing = pd.read_excel(r'data/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_existing')
production_potential = pd.read_excel(r'data/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_potential')
production_existing = production_existing[production_existing['Scenario']=='Domestic demand scenario']
production_potential = production_potential[production_potential['Scenario']=='Domestic demand scenario']

In [ ]:
production_existing['Mineral'].unique()

In [ ]:
specific_lca = specific_lca.reset_index().rename(columns={'index': 'Mineral'})
specific_lca

In [ ]:
# Define a manual mapping to link the mineral in the scenario with the LCI of our choice
mineral_to_material = {
    "Cobalt": "Cobalt",
    "Copper": "Copper concentrate, Canada",
    "Lithium": "Lithium carbonate, spodumene",
    "Nickel": "Nickel",
    "Graphite": "Graphite, natural",
    "Neodymium": "Copper concentrate, Canada"  
}


In [ ]:
# Generate the projected impacts DataFrame
projected_impacts_existing_production = calculate_projected_impacts(production_existing, specific_lca, mineral_to_material)
projected_impacts_potential_production = calculate_projected_impacts(production_potential, specific_lca, mineral_to_material)


In [ ]:
projected_impacts_existing_production

In [ ]:
# Plot impacts per production scenario
plot_impacts_with_production(projected_impacts_existing_production, production_existing, impact_categories, scenario_name='existing_production')
plot_impacts_with_production(projected_impacts_potential_production, production_potential, impact_categories, scenario_name='potential_production')


In [ ]:
# Plot incremental impacts from potential production to existing production
plot_incremental_impacts(projected_impacts_existing_production, projected_impacts_potential_production,
                                 production_existing, production_potential, 
                                 impact_categories, save_dir="results", scenario_name="incremental_comparison")
